In [1]:
!pip install --upgrade -q https://pypi.johnsnowlabs.com/6.0.0rc2-xxxxxxxxxxxxx/spark-ocr/spark_ocr-6.0.0rc2-py3-none-any.whl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.7/635.7 kB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 99.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 128.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 99.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.9 MB/s eta 0:00:00


In [2]:
!pip install --upgrade -q https://pypi.johnsnowlabs.com/5.5.3.PR-2619.yyyyyyyyyyyyyyyyyy/spark-nlp-jsl/spark_nlp_jsl-5.5.3-py3-none-any.whl --force-reinstall
!pip install -q spark-nlp==5.5.3
!pip install -q pandas
!pip install -q matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 546.6/546.6 kB 576.9 kB/s eta 0:00:00


In [1]:
from sparkocr import start
import os
import json
import time
import shutil

license = ""

if license and "json" in license:

    with open(license, "r") as creds_in:
        creds = json.loads(creds_in.read())

        for key in creds.keys():
            os.environ[key] = creds[key]
else:
    raise Exception("License JSON File is not specified")

In [2]:
extra_configurations = {
    "spark.extraListeners": "com.johnsnowlabs.license.LicenseLifeCycleManager",
    "spark.sql.legacy.allowUntypedScalaUDF" : "true",
    "spark.driver.memory": "200g"
}

spark = start(secret="6.0.0rc2-xxxxxxxxxxxxx",
              nlp_secret="5.5.3.PR-2619.yyyyyyyyyyyyyyyyyy",
              nlp_internal=True,
              nlp_jsl=True,
              nlp_version="5.5.3",
              extra_conf=extra_configurations)

spark

Spark version: 3.5.0
Spark NLP version: 5.5.3
Spark NLP for Healthcare version: 5.5.3
Spark OCR version: 6.0.0rc2



In [3]:
import numpy as np
import pandas as pd
import os

#Pyspark Imports
from pyspark.sql import SparkSession
from pyspark.ml import PipelineModel
from pyspark.sql import functions as F

# Necessary imports from Spark OCR library
import sparkocr
from sparkocr import start
from sparkocr.transformers import *
from sparkocr.enums import *
from sparkocr.utils import display_images, to_pil_image, display_images_horizontal
from sparkocr.metrics import score
import pkg_resources

# import sparknlp packages
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp_jsl
from sparknlp_jsl.annotator import *
from collections import Counter
from sparknlp.pretrained import PretrainedPipeline

In [22]:
def evaluate_predictions(SOURCE_GT_PATH, DF_SAVE_PATH, SAVE_MAPPING_PATH):

    def calculate_metrics(preds, gts):
      gt_counter = Counter(gts)
      pred_counter = Counter(preds)

      tp = 0
      for item in pred_counter:
          if item in gt_counter:
              tp += min(pred_counter[item], gt_counter[item])

      fp = sum(pred_counter.values()) - tp
      fn = sum(gt_counter.values()) - tp

      precision = tp / (tp + fp) if (tp + fp) else 0
      recall = tp / (tp + fn) if (tp + fn) else 0

      return precision, recall

    with open(SOURCE_GT_PATH, "r") as f:
        ground_truth = json.load(f)

    df_predictions = spark.read.format("parquet").load(DF_SAVE_PATH)

    predictions_by_file = {}

    for row in df_predictions.select("path").distinct().toLocalIterator():
        file_path = row.asDict()["path"]
        filename = os.path.basename(file_path)

        if filename not in ground_truth:
            continue

        extracted_results = []
        rows = df_predictions.filter(F.col("path") == file_path).select("positions_ner")

        for r in rows.toLocalIterator():
            for ner in r.asDict()["positions_ner"]:
                extracted_results.append(ner.asDict()["result"])

        predictions_by_file[filename] = extracted_results

    summary = {}
    all_precisions = []
    all_recalls = []

    for filename, predictions in predictions_by_file.items():
        gt_values = ground_truth[filename]
        precision, recall = calculate_metrics(predictions, gt_values)

        all_precisions.append(precision)
        all_recalls.append(recall)

        summary[filename] = {
            "precision": round(precision, 4),
            "recall": round(recall, 4),
            "gt": gt_values,
            "pred": predictions
        }

        print(f"Filename: {filename} | Precision: {precision:.4f} | Recall: {recall:.4f}")

    avg_precision = round(sum(all_precisions) / len(all_precisions), 4)
    avg_recall = round(sum(all_recalls) / len(all_recalls), 4)
    f1_score = round(2 * (avg_precision * avg_recall) / (avg_precision + avg_recall), 4)

    print(f"\nOverall Precision: {avg_precision}")
    print(f"Overall Recall: {avg_recall}")
    print(f"F1 Score: {f1_score}")

    with open(SAVE_MAPPING_PATH, "w") as f:
        json.dump(summary, f, indent=4)

    print(f"Mapping File Saved To : {SAVE_MAPPING_PATH}")

In [5]:
ner_threshold = 0.95
ocr_threshold = 70
whitelist = ['HOSPITAL', 'NAME', 'DOCTOR', 'PATIENT', 'AGE', 'ID', 'MEDICALRECORD', 'IDNUM', 'COUNTRY', 'LOCATION', 'STREET', 'STATE', 'ZIP', 'CONTACT', 'PHONE', 'DATE']
matcherWhitelist = {i : 0.95 for i in whitelist}
matcherWhitelist

{'HOSPITAL': 0.95,
 'NAME': 0.95,
 'DOCTOR': 0.95,
 'PATIENT': 0.95,
 'AGE': 0.95,
 'ID': 0.95,
 'MEDICALRECORD': 0.95,
 'IDNUM': 0.95,
 'COUNTRY': 0.95,
 'LOCATION': 0.95,
 'STREET': 0.95,
 'STATE': 0.95,
 'ZIP': 0.95,
 'CONTACT': 0.95,
 'PHONE': 0.95,
 'DATE': 0.95}

In [6]:
pdf_to_image = PdfToImage() \
    .setInputCol("content") \
    .setOutputCol("image_raw") \
    .setPageNumCol("pagenum") \
    .setResolution(400) \
    .setSplitNumBatch(10) \
    .setKeepInput(False) \
    .setImageType(ImageType.TYPE_3BYTE_BGR)

ocr = ImageToText() \
    .setInputCol("image_raw") \
    .setOutputCol("text") \
    .setIgnoreResolution(False) \
    .setPageIteratorLevel(PageIteratorLevel.SYMBOL) \
    .setPageSegMode(PageSegmentationMode.SPARSE_TEXT) \
    .setWithSpaces(True) \
    .setKeepLayout(False) \
    .setConfidenceThreshold(ocr_threshold)

document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document") \
    .setCleanupMode("shrink_full")

sentence_detector = SentenceDetector() \
    .setInputCols(["document"]) \
    .setOutputCol("sentence")

tokenizer = Tokenizer() \
    .setInputCols(["sentence"]) \
    .setOutputCol("token")

word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")

glove_embeddings = WordEmbeddingsModel.pretrained('glove_100d') \
      .setInputCols(['sentence', 'token']) \
      .setOutputCol('glove_embeddings')

ner_deid_subentity_glove_medical = MedicalNerModel.pretrained("ner_deid_subentity_glove", "en", "clinical/models") \
      .setInputCols(["sentence", "token", "glove_embeddings"]) \
      .setOutputCol("ner_deid_subentity_glove_medical")

ner_deid_subentity_glove_converter = NerConverterInternalModel()\
    .setInputCols(["sentence", "token", "ner_deid_subentity_glove_medical"])\
    .setOutputCol("ner_deid_subentity_glove_converter") \
    .setThreshold(ner_threshold) \
    .setWhiteList(whitelist)

ner_deid_generic_augmented_medical = MedicalNerModel.pretrained("ner_deid_generic_augmented", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner_deid_generic_augmented_medical")

ner_deid_generic_augmented_converter = NerConverterInternalModel()\
    .setInputCols(["sentence", "token", "ner_deid_generic_augmented_medical"])\
    .setOutputCol("ner_deid_generic_augmented_converter") \
    .setThreshold(ner_threshold) \
    .setWhiteList(whitelist)

ner_deidentify_dl_medical = MedicalNerModel.pretrained("ner_deidentify_dl","en","clinical/models") \
    .setInputCols("sentence","token","embeddings") \
    .setOutputCol("ner_deidentify_dl_medical")

ner_deidentify_dl_converter = NerConverterInternalModel()\
    .setInputCols(["sentence", "token", "ner_deidentify_dl_medical"])\
    .setOutputCol("ner_deidentify_dl_converter") \
    .setThreshold(ner_threshold)\
    .setWhiteList(whitelist)

ner_deid_enriched_medical = MedicalNerModel.pretrained("ner_deid_enriched","en","clinical/models")\
    .setInputCols(["sentence","token","embeddings"])\
    .setOutputCol("ner_deid_enriched_medical")

ner_deid_enriched_converter = NerConverterInternalModel()\
    .setInputCols(["sentence", "token", "ner_deid_enriched_medical"])\
    .setOutputCol("ner_deid_enriched_converter") \
    .setThreshold(ner_threshold) \
    .setWhiteList(whitelist)

regex_matcher = RegexMatcher()\
    .setInputCols("sentence")\
    .setOutputCol("regex_chunk")\
    .setRules(["(0[1-9]|1[0-2])/(0[1-9]|[12][0-9]|3[01])/\d{4};DATE",
               "DR[A-Za-z0-9]{5,6};ID"])\
    .setDelimiter(";")

chunk_merger = ChunkMergeApproach() \
    .setInputCols('regex_chunk', 'ner_deid_subentity_glove_converter', 'ner_deid_generic_augmented_converter', "ner_deidentify_dl_converter", "ner_deid_enriched_converter") \
    .setOutputCol('merged_ner_chunk') \
    .setMergeOverlapping(True)

deid_obfuscated = DeIdentification() \
    .setInputCols(["sentence", "token", "merged_ner_chunk"]) \
    .setOutputCol("obfuscated") \
    .setMode("obfuscate") \
    .setKeepMonth(True) \
    .setKeepYear(True) \
    .setObfuscateDate(True) \
    .setSameEntityThreshold(0.7) \
    .setKeepTextSizeForObfuscation(True) \
    .setFakerLengthOffset(2) \
    .setReturnEntityMappings(True) \
    .setDays(2) \
    .setMappingsColumn("aux") \
    .setIgnoreRegex(False) \
    .setGroupByCol("path") \
    .setRegion("us") \
    .setSeed(40) \
    .setConsistentObfuscation(True) \
    .setChunkMatching(matcherWhitelist)

cleaner = NerOutputCleaner() \
    .setInputCol("aux") \
    .setOutputCol("new_aux") \
    .setOutputNerCol("positions_ner")

position_finder = PositionFinder() \
    .setInputCols("positions_ner") \
    .setOutputCol("coordinates") \
    .setPageMatrixCol("positions")

drawRegions = ImageDrawRegions()  \
    .setInputCol("image_raw")  \
    .setInputChunkMappingCol("new_aux") \
    .setInputRegionsCol("coordinates")  \
    .setOutputCol("image_with_regions")  \
    .setPatchImages(True) \
    .setUseContrastTextColor(False) \
    .setPageMatrixCol("positions")

stages = [
    pdf_to_image,
    ocr,
    document_assembler,
    sentence_detector,
    tokenizer,
    word_embeddings,
    glove_embeddings,
    ner_deid_subentity_glove_medical,
    ner_deid_subentity_glove_converter,
    ner_deid_generic_augmented_medical,
    ner_deid_generic_augmented_converter,
    ner_deidentify_dl_medical,
    ner_deidentify_dl_converter,
    ner_deid_enriched_medical,
    ner_deid_enriched_converter,
    regex_matcher,
    chunk_merger,
    deid_obfuscated,
    cleaner,
    position_finder,
    drawRegions
]

obfuscation_pipe = Pipeline(stages=stages)

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[OK!]
ner_deid_subentity_glove download started this may take some time.
[OK!]
ner_deid_generic_augmented download started this may take some time.
[OK!]
ner_deidentify_dl download started this may take some time.
[OK!]
ner_deid_enriched download started this may take some time.
[OK!]


In [38]:
SOURCE_PDF_PATH = "/content/PDF_FILES_EASY/*"
DF_SAVE_PATH = "/content/easy/"
SOURCE_GT_PATH = "/content/pdf_deid_gts_easy.json"
SAVE_MAPPING_PATH = "/content/easy_result_mapping.json"
SAVE_OUTPUT_PDF = "/content/easy_pdf_output/"

os.makedirs(SAVE_OUTPUT_PDF, exist_ok=True)

In [8]:
df = spark.read.format('binaryFile').load(SOURCE_PDF_PATH)
result = obfuscation_pipe.fit(df).transform(df)
result.write.format("parquet").mode("overwrite").save(DF_SAVE_PATH)

In [39]:
evaluate_predictions(SOURCE_GT_PATH=SOURCE_GT_PATH, DF_SAVE_PATH=DF_SAVE_PATH, SAVE_MAPPING_PATH=SAVE_MAPPING_PATH)

Filename: PDF_Deid_Deidentification_11.pdf | Precision: 0.9737 | Recall: 0.9024
Filename: PDF_Deid_Deidentification_5.pdf | Precision: 0.9268 | Recall: 0.9268
Filename: PDF_Deid_Deidentification_15.pdf | Precision: 0.8750 | Recall: 0.8537
Filename: PDF_Deid_Deidentification_9.pdf | Precision: 1.0000 | Recall: 1.0000
Filename: PDF_Deid_Deidentification_3.pdf | Precision: 0.9268 | Recall: 0.9268
Filename: PDF_Deid_Deidentification_27.pdf | Precision: 0.8750 | Recall: 0.8537
Filename: PDF_Deid_Deidentification_7.pdf | Precision: 0.9268 | Recall: 0.9268
Filename: PDF_Deid_Deidentification_22.pdf | Precision: 1.0000 | Recall: 1.0000
Filename: PDF_Deid_Deidentification_8.pdf | Precision: 0.9512 | Recall: 0.9512
Filename: PDF_Deid_Deidentification_10.pdf | Precision: 1.0000 | Recall: 0.9756
Filename: PDF_Deid_Deidentification_1.pdf | Precision: 0.8750 | Recall: 0.8537
Filename: PDF_Deid_Deidentification_17.pdf | Precision: 0.9474 | Recall: 0.8780
Filename: PDF_Deid_Deidentification_21.pdf | P

In [30]:
OBFUSCATED_IMAGE_COL = "image_with_regions"

img_to_pdf = ImageToPdf() \
    .setPageNumCol("pagenum") \
    .setOriginCol("path") \
    .setOutputCol("pdf") \
    .setInputCol(OBFUSCATED_IMAGE_COL) \
    .setAggregatePages(True)

source = spark.read.format("parquet").load(DF_SAVE_PATH)
result_pdf = img_to_pdf.transform(source)

for row in result_pdf.select("path", "pdf").toLocalIterator():
  filename = row.asDict()["path"]
  basename = os.path.basename(filename)

  savename = os.path.join(SAVE_OUTPUT_PDF, basename)

  pdfFile = open(savename, "wb")
  pdfFile.write(row.asDict()["pdf"])
  pdfFile.close()

In [41]:
SOURCE_PDF_PATH = "/content/PDF_FILES_MEDIUM/*"
DF_SAVE_PATH = "/content/medium/"
SOURCE_GT_PATH = "/content/pdf_deid_gts_medium.json"
SAVE_MAPPING_PATH = "/content/medium_result_mapping.json"
SAVE_OUTPUT_PDF = "/content/medium_pdf_output/"

os.makedirs(SAVE_OUTPUT_PDF, exist_ok=True)

In [10]:
df = spark.read.format('binaryFile').load(SOURCE_PDF_PATH)
result = obfuscation_pipe.fit(df).transform(df)
result.write.format("parquet").mode("overwrite").save(DF_SAVE_PATH)

In [42]:
evaluate_predictions(SOURCE_GT_PATH=SOURCE_GT_PATH, DF_SAVE_PATH=DF_SAVE_PATH, SAVE_MAPPING_PATH=SAVE_MAPPING_PATH)

Filename: PDF_Deid_Deidentification_Adv_4.pdf | Precision: 0.9545 | Recall: 0.8077
Filename: PDF_Deid_Deidentification_Adv_3.pdf | Precision: 0.9111 | Recall: 0.7885
Filename: PDF_Deid_Deidentification_12.pdf | Precision: 0.9231 | Recall: 0.9231
Filename: PDF_Deid_Deidentification_6.pdf | Precision: 0.9512 | Recall: 0.9512
Filename: PDF_Deid_Deidentification_11.pdf | Precision: 0.9737 | Recall: 0.9024
Filename: PDF_Deid_Deidentification_Adv_2.pdf | Precision: 0.9268 | Recall: 0.7308
Filename: PDF_Deid_Deidentification_3.pdf | Precision: 0.9268 | Recall: 0.9268
Filename: PDF_Deid_Deidentification_Adv_9.pdf | Precision: 0.9545 | Recall: 0.8077
Filename: PDF_Deid_Deidentification_29.pdf | Precision: 0.9487 | Recall: 0.9487
Filename: PDF_Deid_Deidentification_5.pdf | Precision: 0.9268 | Recall: 0.9268
Filename: PDF_Deid_Deidentification_Adv_5.pdf | Precision: 0.9070 | Recall: 0.7500
Filename: PDF_Deid_Deidentification_13.pdf | Precision: 0.9512 | Recall: 0.9512
Filename: PDF_Deid_Deidentif

In [32]:
OBFUSCATED_IMAGE_COL = "image_with_regions"

img_to_pdf = ImageToPdf() \
    .setPageNumCol("pagenum") \
    .setOriginCol("path") \
    .setOutputCol("pdf") \
    .setInputCol(OBFUSCATED_IMAGE_COL) \
    .setAggregatePages(True)

source = spark.read.format("parquet").load(DF_SAVE_PATH)
result_pdf = img_to_pdf.transform(source)

for row in result_pdf.select("path", "pdf").toLocalIterator():
  filename = row.asDict()["path"]
  basename = os.path.basename(filename)

  savename = os.path.join(SAVE_OUTPUT_PDF, basename)
  if "Adv" in basename:
    pdfFile = open(savename, "wb")
    pdfFile.write(row.asDict()["pdf"])
    pdfFile.close()